# Part 1: 오픈소스 Literature Agent 개관

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. 오픈소스 문헌 리뷰 에이전트의 종류와 특징 이해
2. 각 에이전트의 핵심 구성요소 파악
3. 학술 API (Semantic Scholar, arXiv, PubMed) 기본 사용법

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
# Colab: 자동 설치
# 로컬: pyproject.toml로 미리 설치 필요 (uv pip install -e . 또는 pip install -e .)

if IN_COLAB:
    print("📦 패키지 설치 중...")
    !pip install requests python-dotenv semanticscholar arxiv google-generativeai -q
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")

In [ ]:
# Cell 3: API Key 로딩
import os
from pathlib import Path

# Colab 환경 체크
if IN_COLAB:
    # Colab userdata에서 키 불러오기
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        print("⚠️ Colab Secrets에 GEMINI_API_KEY를 추가하세요")
        print("   좌측 🔑 아이콘 > Add new secret")
        GEMINI_API_KEY = None
else:
    # 로컬 환경: dotenv 사용
    try:
        from dotenv import load_dotenv
        # 현재 디렉토리에서 .env 찾기
        load_dotenv()
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        
        if GEMINI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 GEMINI_API_KEY를 추가하세요")
            print("   예: GEMINI_API_KEY=your_key_here")
    except ImportError:
        print("⚠️ python-dotenv가 설치되지 않았습니다")
        print("   실행: pip install python-dotenv")
        GEMINI_API_KEY = None

# API 사용 가능 여부 확인
if GEMINI_API_KEY:
    print(f"🔑 API Key: {GEMINI_API_KEY[:10]}...")
else:
    print("❌ API Key를 설정하지 않으면 일부 기능이 동작하지 않습니다")
    
print("\n💡 Semantic Scholar, arXiv, PubMed는 API key 없이도 사용 가능합니다")

---
## 1. 파악하기: 오픈소스 Literature Agent 비교

코드가 공개된 에이전트만 다룹니다. 웹 서비스(Elicit, Consensus 등)는 제외.

### 주요 오픈소스 프로젝트

| 프로젝트 | Stars | 용도 | 검색 DB |
|----------|------:|------|--------|
| [GPT-Researcher](https://github.com/assafelovic/gpt-researcher) | 24.9k | 웹검색 → 보고서 | 웹 (Tavily) |
| [AI-Scientist](https://github.com/SakanaAI/AI-Scientist) | 12k | 아이디어→논문 자동화 | Semantic Scholar |
| [PaperQA2](https://github.com/Future-House/paper-qa) | 8k | PDF RAG Q&A | Semantic Scholar |
| [**Agent Laboratory**](https://github.com/SamuelSchmidgall/AgentLaboratory) | 5.2k | **문헌→실험→보고서** | **arXiv** |
| [**PaSa**](https://github.com/bytedance/pasa) | 1.5k | **논문 검색 특화** | **arXiv + Scholar** |

### 오늘 집중할 4가지 도구

| 도구 | 기능 | 저장소 |
|------|------|--------|
| **Agent Laboratory** | 문헌 검색 → 실험 → 보고서 | github.com/SamuelSchmidgall/AgentLaboratory |
| **PaSa** | 논문 검색 특화 (PPO 학습) | github.com/bytedance/pasa |
| **LitLLM** | Related Work 자동 생성 | github.com/ServiceNow/litllm |
| **AgentReview** | Peer Review 시뮬레이션 | github.com/ahren09/agentreview |

In [ ]:
# Cell 6: 각 도구의 핵심 구조 확인
print("""
📁 Agent Laboratory 구조
├── agents/
│   ├── literature_agent.py   # 문헌 검색
│   ├── experiment_agent.py   # 실험 설계
│   └── report_agent.py       # 보고서 생성
└── tools/arxiv_search.py     # arXiv API

📁 PaSa 구조
├── agents/
│   ├── crawler.py    # 논문 크롤링
│   └── selector.py   # 관련성 평가 (PPO)
└── search/
    ├── arxiv.py      # arXiv 검색
    └── scholar.py    # Google Scholar

📁 LitLLM 구조
├── retriever.py      # 관련 논문 검색
├── summarizer.py     # 논문 요약
└── writer.py         # Related Work 생성

📁 AgentReview 구조
├── arena.py          # 시뮬레이션 루프
├── paper.py          # 논문 파싱
└── reviewer.py       # 리뷰어 에이전트
""")

### 공통 구성요소

모든 Literature Agent는 다음 3가지 핵심 요소로 구성됩니다:

1. **검색 모듈** - 논문 DB에서 관련 논문 찾기
2. **처리 모듈** - 논문 파싱, 요약, 분석
3. **생성 모듈** - 결과물 생성 (리뷰, 보고서 등)

### 질문

이 구조를 보고 생각해보세요:
- 본인 연구에서 가장 자주 쓸 것 같은 기능은?
- 어떤 부분을 커스텀하면 유용할까?

---
## 2. 써보기: 학술 API 기본 사용법

### API 비교

| API | Key 필요 | 비용 | 특징 |
|-----|----------|------|------|
| **Semantic Scholar** | 선택 | 무료 | 인용 네트워크, 임베딩 |
| **arXiv** | 불필요 | 무료 | 프리프린트, 전문 접근 |
| **PubMed** | 불필요 | 무료 | 의학/심리학 특화 |

In [ ]:
# Cell 10: Semantic Scholar API 테스트
import requests

def search_semantic_scholar(query, limit=5):
    """Semantic Scholar에서 논문 검색"""
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": limit,
        "fields": "title,authors,year,abstract,citationCount"
    }
    response = requests.get(url, params=params)
    return response.json()

# 테스트: LLM 관련 논문 검색
results = search_semantic_scholar("large language model psychology")
print(f"검색 결과: {len(results.get('data', []))}개 논문\n")

for paper in results.get('data', [])[:3]:
    print(f"📄 {paper['title']}")
    print(f"   - 연도: {paper.get('year', 'N/A')}, 인용: {paper.get('citationCount', 0)}")
    print()

In [ ]:
# Cell 11: arXiv API 테스트
import arxiv

def search_arxiv(query, max_results=5):
    """arXiv에서 논문 검색"""
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    return list(search.results())

# 테스트
papers = search_arxiv("cognitive psychology AI")
print(f"arXiv 검색 결과: {len(papers)}개\n")

for paper in papers[:3]:
    print(f"📄 {paper.title}")
    print(f"   - 날짜: {paper.published.strftime('%Y-%m-%d')}")
    print(f"   - PDF: {paper.pdf_url}")
    print()

### DIY: 본인 연구 주제로 검색해보세요

In [ ]:
# DIY Cell 13: 본인 연구 주제로 검색
# 힌트: query 변수만 바꾸면 됩니다

my_query = "your research topic here"  # <- 이 부분 수정!

# Semantic Scholar 검색
ss_results = search_semantic_scholar(my_query, limit=5)
print("=== Semantic Scholar 결과 ===")
for paper in ss_results.get('data', []):
    print(f"- {paper['title']} ({paper.get('year', 'N/A')})")

print("\n=== arXiv 결과 ===")
arxiv_results = search_arxiv(my_query, max_results=5)
for paper in arxiv_results:
    print(f"- {paper.title}")

In [ ]:
# DIY Cell 14: 검색 결과 분석
# 어떤 DB가 더 관련성 높은 결과를 주나요?

# 메모:
# - Semantic Scholar: 
# - arXiv: 

---
## 3. 바꿔보기: 검색 함수 커스텀

### 수정 포인트

| 항목 | 기본값 | 커스텀 아이디어 |
|------|--------|----------------|
| 검색 필드 | title, abstract | + authors, venue |
| 정렬 기준 | relevance | citationCount, year |
| 필터 | 없음 | 연도 범위, 최소 인용수 |

In [ ]:
# Cell 16: 커스텀 검색 함수 예시

def search_semantic_scholar_advanced(query, limit=10, year_from=2020, min_citations=10):
    """필터링 옵션이 추가된 검색 함수"""
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": limit,
        "fields": "title,authors,year,abstract,citationCount,venue",
        "year": f"{year_from}-"  # 2020년 이후만
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # 인용수 필터링
    filtered = [
        p for p in data.get('data', [])
        if p.get('citationCount', 0) >= min_citations
    ]
    return filtered

# 테스트
results = search_semantic_scholar_advanced("LLM reasoning", year_from=2023, min_citations=50)
print(f"필터링 후: {len(results)}개 논문")
for p in results[:3]:
    print(f"- {p['title']} (인용: {p['citationCount']})")

In [ ]:
# DIY Cell 17: 본인만의 필터 추가하기
# 예: 특정 저자 필터, 특정 저널 필터 등

def my_custom_search(query):
    # 여기에 코드 작성
    pass

In [ ]:
# Cell 18: PubMed API 추가 예시 (심리학 연구용)
import requests

def search_pubmed(query, max_results=5):
    """PubMed에서 논문 검색 (심리학/의학 특화)"""
    # E-utilities API
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    
    # 1. 검색
    search_url = f"{base_url}esearch.fcgi"
    search_params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "json"
    }
    search_result = requests.get(search_url, params=search_params).json()
    ids = search_result.get('esearchresult', {}).get('idlist', [])
    
    if not ids:
        return []
    
    # 2. 상세 정보
    fetch_url = f"{base_url}esummary.fcgi"
    fetch_params = {
        "db": "pubmed",
        "id": ",".join(ids),
        "retmode": "json"
    }
    fetch_result = requests.get(fetch_url, params=fetch_params).json()
    
    papers = []
    for pid in ids:
        info = fetch_result.get('result', {}).get(pid, {})
        papers.append({
            'title': info.get('title', ''),
            'authors': [a.get('name', '') for a in info.get('authors', [])],
            'year': info.get('pubdate', '')[:4],
            'journal': info.get('source', '')
        })
    return papers

# 테스트
pubmed_results = search_pubmed("cognitive behavioral therapy depression")
print(f"PubMed 결과: {len(pubmed_results)}개")
for p in pubmed_results[:3]:
    print(f"- {p['title'][:60]}... ({p['year']})")

In [ ]:
# DIY Cell 19: 세 DB 결과 비교
comparison_query = "your topic"  # <- 수정

# 세 DB에서 검색
ss = search_semantic_scholar(comparison_query)
ax = search_arxiv(comparison_query)
pm = search_pubmed(comparison_query)

print(f"Semantic Scholar: {len(ss.get('data', []))}개")
print(f"arXiv: {len(ax)}개")
print(f"PubMed: {len(pm)}개")

In [ ]:
# Cell 20: 결과 비교 분석
print("""
📊 DB별 특징 비교

| DB | 장점 | 단점 | 추천 용도 |
|----|------|------|----------|
| Semantic Scholar | 인용 네트워크, 임베딩 | rate limit | 인용 분석 |
| arXiv | 최신 논문, 전문 무료 | CS 편중 | 프리프린트 |
| PubMed | 의학/심리학 특화 | 일반 CS 없음 | 심리학 논문 |
""")

---
## 4. 토론

### 토론 질문

1. **본인 연구에 가장 유용할 도구는?**
   - Agent Laboratory, PaSa, LitLLM, AgentReview 중

2. **어떤 API 조합이 좋을까?**
   - 심리학 연구: PubMed + Semantic Scholar?
   - AI 연구: arXiv + Semantic Scholar?

3. **커스텀하고 싶은 기능은?**
   - 한글 논문 검색?
   - 특정 저널 필터?
   - 메타분석용 효과크기 추출?

### 다음 단계

이어서 실습할 내용:
- **Part 2**: Agent Laboratory / PaSa로 문헌 검색 자동화
- **Part 3**: LitLLM으로 Related Work 초안 생성
- **Part 4**: AgentReview로 논문 피드백 받기

각 노트북에서 **파악하기 → 써보기 → 바꿔보기** 패턴을 따릅니다.